# **Assignment 2: N-gram Language Models**

Due: 5/6 (Thursday) 11:59pm on Canvas. Please work in a group of 2 on this assignment. 

In this assignment, which is based on [Chapter 3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) of Jurasky and Martin's [book](https://web.stanford.edu/~jurafsky/slp3/), you will get practicallly acquinted with n-gram language models, one of the simplest and yet most useful tools in the language technologiest toolbox. You will extract n-gram statistics from text and to use these statistics for language modeling and generation. 

**The learning goals of this assignment are**:
- Experiment with language models and implement basic smoothing.
- Have fun using language models to probabilistically generate sentences
- Understand how to evaluate your language models by computing the perplexity. 

**Writing Code**

Look for the keyword "TODO" and fill in your code in the empty space. Include the output in your code. 


**Submission**
- link to your code file (**include output**. same as previoius assignments)

The submission link will be closed at 11:59pm 5/6 (Thursday). No late submission will be accepted.

**This exercise will take 12% of the total final grade.** If you wish to discuss any of it, feel free to bring it to the instructor during office hours. 


**Attention:** DO NOT edit this file. All your changes to this file cannot be saved and will get lost. Make your own copy of this file for editing.

See the instruction from the previoius assignment for making a copy of this file. 


0-1. Mount Google Drive

You need to mount Google drive first on Google Colab space so you can access the files in your Google drive. When you do so, you will be asked to go to an URL to retrieve an authorization code and enter it here.

Goal in thie following cell: load the Drive helper and mount

It will show Drive mounted at /content/gdrive after mounted.

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
# explaination of drive.mount()
# This will prompt for authorization for accessing the data on your Google Drive
# You need to (i) click the link showed after execute, (ii) copy the code, 
# (iii) paste below and (iv) press "Enter"

Mounted at /content/gdrive


# **Training data and test data**

For this assignment, you are provided a [training data set](https://drive.google.com/file/d/1_kmHPZO6Me3arN_XtDg8CKWQvDxUCuf5/view?usp=sharing) and a [test data set](https://drive.google.com/file/d/1oaA1q1yJ9YD025-sqRm-5UCo5crYqnuk/view?usp=sharing).
You will notice they have one sentence per line, and have been tokenized with punctuations removed.
You need to surround each sentence by a start of sentence and end of sentence marker (`<s>` and `</s>`).
No need to further process the corpus unless you are asked to. Use the training data to build N-gram models. 




# **Task 0 - Process the training data**

In [ ]:
#TODO: import the needed libraries, 
#padding each sentence with <s> at the beginning and </s> at the end
#you may want to put the padded training data into a list for the convenience of subsequent steps

import nltk
nltk.download('punkt')
from collections import Counter, defaultdict
import random
import heapq
import operator
import math


## Loads in the words and labels of one of the datasets
def load_file(data_file):
    sentences = []   
    with open(data_file, 'rt', encoding="utf8") as f:
        i = 0
        for line in f:
            sentences.append("<s> " + line + " </s>")
    return sentences

# file paths
cwd = "/content/gdrive/MyDrive/1 - School/1 - SP21/CSCI 404/Assignment 2/"

training_file = cwd + "data/train.txt"
test_file = cwd + "data/test.txt"


sentences = load_file(training_file)



# See References section at the end of this document
def train_ngram(ngrams):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for ng in ngrams:
        model[ng[:-1]][ng[-1]] += 1  # [(w1, w2, ... , wn-1)][wn]
    
    total_count = 0      
    for hist in model: # hist = (w1, w2, ... , wn-1)
        total_count = float(sum(model[hist].values()))
        for w in model[hist]: # w = wn
            model[hist][w] /= total_count
    
    return model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#**Task 1 - Generate n-grams from text** 
You may find the online resource [1](https://www.programcreek.com/python/example/86315/nltk.bigrams) helpful for using NLTK to extract n-grams and their statistics. 

**1.1 Extract unigrams from training data**




In [ ]:
#TODO: extract unigrams from training data,
# output top 50 most frequent unigrams (sounds familiar? yes, we did something similar in assignment 0) and their probabilities 

unigrams = [ug for sent in sentences for ug in sent.split()]

unigram_fdist = nltk.FreqDist(unigrams)

unigram_count = len(unigrams)
for ug, freq in unigram_fdist.most_common(50): 
    print(f"{str(ug):<15} {freq/unigram_count}")


the             0.046396693775689356
<s>             0.04087814429576155
</s>            0.04087814429576155
of              0.02472991469412588
to              0.02379107998013322
in              0.018507579829906043
and             0.017952999672293544
said            0.01682953200656503
a               0.016813862051251655
mln             0.011665941079605417
for             0.009064047195180194
dlrs            0.008120443364353032
vs              0.007922184364518588
it              0.007125060550751238
pct             0.006801441908409793
on              0.006238686121938142
is              0.0056425465176249525
from            0.005237852889096914
its             0.00499667183775192
that            0.004994627930537132
at              0.00493262941168856
by              0.004845422703857603
be              0.004775248556149878
cts             0.004544287040878826
year            0.004524529271135874
will            0.004520441456706298
with            0.0043569288795232515
billio

**1.2 Extract bigrams from training data**

In [ ]:
#TODO: extract bigrams from training data, 
#output top 50 most frequent bigrams and their probabilities


bigrams = [bg for sent in sentences for bg in nltk.bigrams(sent.split())]

bigram_fdist = nltk.FreqDist(bigrams)

bg_count = len(bigrams)
for bg, freq in bigram_fdist.most_common(50): 
    probability = freq / unigram_fdist[bg[0]] # P(bg[1]|bg[0]) = count(bg) / count(bg[0])
    print(f"{str(bg):<30} {probability}")

('<s>', 'the')                 0.18155
('said', '</s>')               0.31503521982025745
('of', 'the')                  0.18675960107994932
('in', 'the')                  0.22731455917540955
('mln', 'dlrs')                0.26262921217076446
('said', 'it')                 0.1644401263055623
('said', 'the')                0.144887053679864
('mln', 'vs')                  0.1872335455235648
('for', 'the')                 0.20114251352976548
('cts', 'vs')                  0.40104947526236884
('the', 'company')             0.03596182085168869
('to', 'the')                  0.06867124856815579
('he', 'said')                 0.4872538860103627
('will', 'be')                 0.34800301431801056
('on', 'the')                  0.22649339303265262
('billion', 'dlrs')            0.31903302689819546
('the', 'us')                  0.026784140969162994
('<s>', 'it')                  0.030116666666666667
('cts', 'net')                 0.26401799100449774
('to', 'be')                   0.0460481099656

**1.3 Extract trigrams from training data**

In [ ]:
#TODO: extract trigrams from training data, 
#output top 50 most frequent trigrams and their probabilities
#this will require re-padding each sentence with <s><s> at the beginning 
#that is, append extra <s> to the beginning of each sentence


trigrams = [tg for sent in sentences for tg in nltk.trigrams(["<s>"] + sent.split())]

trigram_fdist = nltk.FreqDist(trigrams)

# condition_pairs = (((w0, w1), w2) for w0, w1, w2 in trigrams)
# trigram_probabilities = nltk.ConditionalFreqDist(condition_pairs)

tg_count = len(trigrams)
num_of_sents = len(sentences)
for tg, freq in trigram_fdist.most_common(50): 
    if tg[:2] == ("<s>","<s>"):
        probability = freq / num_of_sents
    else: 
        probability = freq / bigram_fdist[(tg[0],tg[1])] # P(tg[2]|tg[0],tg[1]) = count(tg) / count(tg[0],tg[1])
    
    print(f"{str(tg):<35} {probability}")

('<s>', '<s>', 'the')               0.18155
('<s>', '<s>', 'it')                0.030116666666666667
('<s>', '<s>', 'shr')               0.02355
('<s>', '<s>', 'he')                0.022183333333333333
('<s>', '<s>', 'in')                0.020916666666666667
('he', 'said', '</s>')              0.4802211824755423
('the', 'company', 'said')          0.4593711719069008
('<s>', 'the', 'company')           0.09547415771596438
('<s>', '<s>', 'but')               0.017283333333333335
('<s>', '<s>', 'a')                 0.016433333333333335
('<s>', 'it', 'said')               0.46873270614277807
('<s>', 'he', 'said')               0.5507137490608565
('mln', 'dlrs', '</s>')             0.1625528129864354
('mln', 'dlrs', 'in')               0.15677118078719146
('<s>', '<s>', 'us')                0.010866666666666667
('qtr', 'net', '</s>')              0.9470499243570348
('inc', 'said', 'it')               0.6775244299674267
('pct', 'of', 'the')                0.3944223107569721
('said', 'it', 'h

# **Task 2 - Random sentence generator using n-grams**

In [ ]:
# randomly generate 10 sentences using each language model, the maximum length of each sentence is set to 30
# suppose all 10 sentences are stored in sent_list, print_sentences() is to print all the sentences
# Feel free to make change to the code in this cell if needed. 
num_sentences = 10
max_sent_len = 30
random.seed()

def print_sentences(sent_list):
    for sent, prob in sent_list:
        for word in sent:
            print(word,end=" ")
        print("\nprobability of: ", prob)
        print("\n\n")


def ngram_random_sentence(model, seed, n=50):
    # how far are you looking back from current word; bigram -> 1, trigram -> 2, etc.
    hist_size = len(seed) * -1

    # max number of tries to get a value that is not <UNK>
    max_tries = 100

    # probability of the sentence
    probability = 1.0

    sentence = seed

    while len(sentence) <= max_sent_len:
        # hist_size number of words before current word
        given = tuple(w for w in sentence[hist_size:])

        # get n most probable next words from the model[given], P(word|given)
        most_probable = heapq.nlargest(n, model[given].items(), key=operator.itemgetter(1))

        # choose one (word, probability) pair
        choice = random.choice(most_probable)

        tries = 0
        while choice[0] == "<UNK>" and tries <= max_tries:
            choice = random.choice(most_probable)
            most_probable = heapq.nlargest(n * 2, model[given].items(), key=operator.itemgetter(1))
            tries += 1

        sentence.append(choice[0])
        probability *= choice[1]

        if sentence[-1] == "</s>":
            break


    if not sentence[-1] == "</s>":
        sentence.append("</s>")
    
    return sentence, probability

def old_ngram_random_sentence(model, seed):
    # how far are you looking back from current word; bigram -> 1, trigram -> 2, etc.
    hist_size = len(seed) * -1
    probability = 1.0
    sentence = seed

    while len(sentence) <= max_sent_len:
        r = random.random()
        accum = 0.0

        # hist_size number of words before current word
        given = tuple(w for w in sentence[hist_size:])


        for word in model[given]: 
            accum += model[given][word] # P(word|given)

            if accum > r:
                probability *= model[given][word] # P(word|given)
                sentence.append(word)
                break

        if sentence[-1] == "</s>":
            break


    if not sentence[-1] == "</s>":
        sentence.append("</s>")
    
    return sentence, probability

**2.1 Generate 10 sentences (with their probabilities) using the unigram model**

In [ ]:
unigram_model = {}
for ug, freq in unigram_fdist.most_common(200): 
    unigram_model[ug] = freq/unigram_count

In [ ]:
unigram_sentences = []

def unigram_random_sentence(unigram_model):
    probability = unigram_model["<s>"]

    sentence = ["<s>"]

    while len(sentence) <= max_sent_len:
        # choose one (word, probability) pair
        choice = random.choice(list(unigram_model.items()))

        while choice[0] == "<s>":
            choice = random.choice(list(unigram_model.items()))
        
        sentence.append(choice[0])
        probability *= choice[1]

        if sentence[-1] == "</s>":
            break

    if not sentence[-1] == "</s>":
        sentence.append("</s>")
        probability *= unigram_model["</s>"]

    return sentence, probability


#generate the sentences
for index in range(0,num_sentences):
    test, probability = unigram_random_sentence(unigram_model)

    unigram_sentences.append((test,probability))
    
#TODO: generate 10 sentences using the unigram model. Suppose maximum sentence length = 30
    
print_sentences(unigram_sentences)

<s> was officials expected industry 1 loss yen other not 25 now </s> 
probability of:  6.339544306575053e-36



<s> per also some exports banks earnings world we prices 1985 other because american month cts international rose be after net offer meeting export current stock international board month if money </s> 
probability of:  1.5864586864867645e-92



<s> may bank week is officials west earnings dlr the major american of as rise net export officials common has we this it sale revs agreement loan world capital us trade </s> 
probability of:  9.293000458771171e-88



<s> by sale sale end against january issue increase all japanese told foreign rose 4 made 20 its increase for japan earlier dlrs were west nine dlr offer co price other </s> 
probability of:  5.29308591647337e-92



<s> price corp have interest 1985 revs foreign board while more total increase american were finance made us this due if made countries march at under were inc dlr foreign this </s> 
probability of:  7.335563

**2.2 Generate 10 sentences (with their probabilities) using the bigram model**

In [ ]:
bigram_model = train_ngram(bigrams)

In [ ]:

bigram_sentences = []

# for word in bigram_model["liberty",]:
#     print(word)

# print()

# test, probability = ngram_random_sentence(bigram_model, ["<s>"])

# print()
# print(test)
# print(probability)

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(bigram_model, ["<s>"])

    bigram_sentences.append((test,probability))
    

#ToDo: generate 10 sentences using the bigram model. Suppse maximum sentence length = 30

print_sentences(bigram_sentences)

<s> as many concessions on tuesday morning delegates saying more on tuesday they agreed yesterday he raised were seen whites market from 25 largest in quarter sales fell 178 154 152 </s> 
probability of:  7.377269879526369e-67



<s> earlier a 1 10 to around march 27 after the group sales gained 26 cts 25 week fell 22 cts higher to make some progress industrial corp to seek protection </s> 
probability of:  9.593528722286043e-65



<s> uk one or if that will receive 92 since november 1982 before the companys capital markets that it expects from 11 elections due for management to be an export quotas </s> 
probability of:  6.64771642855045e-59



<s> the countrys creditors there had to improve says bank was announced program acreage have taken care a definitive documentation japsper said earlier due september 1 0 1 1990 said sales </s> 
probability of:  6.306728320417958e-62



<s> japanese trading of us official who had a 10 25 record in washington said any possible future earnings afte

**2.3 Generate 10 sentences (with their probabilities) using the trigram model**

In [ ]:
trigram_model = train_ngram(trigrams)

In [ ]:
trigram_sentences = []

# for word in trigram_model[("<s>","liberty")]:
#     print(word)


# test, probability = ngram_random_sentence(trigram_model, ["<s>","<s>"])

# print()
# print(test)
# print(probability)

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(trigram_model, ["<s>","<s>"])

    trigram_sentences.append((test,probability))

#ToDo: generate 10 sentences using the trigram model. Suppse maximum sentence length = 30
    
print_sentences(trigram_sentences)

<s> <s> analysts computer industry could charge foreign producers are annoyed the buffer </s> 
probability of:  4.3785214258567445e-11



<s> <s> japanese buyers of non metal mineral ores </s> 
probability of:  1.1022927689594354e-09



<s> <s> but it is likely to happen any time before some operators try to end august 1986 when gross domestic product will be used simultaneously merrill said it does points </s> 
probability of:  9.010259212661037e-30



<s> <s> the report indicated that they soviets will buy the gwinnett daily news severance charges of about 21 mln in principal to foreign banks have largely achieved what we want </s> 
probability of:  4.1034456149783543e-29



<s> <s> its commercial paper </s> 
probability of:  1.3687600644122387e-07



<s> <s> we expect natural gas subsidiary c </s> 
probability of:  8.92857142857143e-09



<s> <s> we pointed out its reaction to what most of first priority for agriculture workers would push to coordinate repayment of all corporate rat

#**Task 3 - Deal with unseen words and smoothing**

The current n-gram models can’t predict the probability of a given sentence if the sentence contains an unknown token or bi-/tri-gram simply because it’s not included in the ngram model from which it looks up corrensponding probability. 

The basic idea of smoothing is to take some probability mass from the words seen during training and reallocate it to words not seen during training. Assume we have decided how much probability mass to reallocate, according to some smoothing scheme. The question is, how do we decide how to allocate this probability mass among unknown words, when we don't even know how many unknown words there are? (No fair peeking at the test data!)

There are multiple approaches, but no perfect solution. (This is an opportunity for you to experiment and innovate.) A straightforward and widely-used approach is to assume a special token `<UNK>` which represents (an equivalence class of) all unknown words. All of the reallocated probability mass is assigned to this special token, and any unknown word encountered during testing is treated as an instance of this token.

This approach has the shortcoming that it treats all unknown words alike, that is, it will assign the same probability to any unknown word. You might think that it's possible to do better than this. Here are two unknown words you might encounter, say, on the internet: "flavodoxin" and "B000EQHXQY". Intuitively, which should be considered more probable? What kind of knowledge are you applying? Do you think a machine could make the same judgment?

A paper by Efron & Thisted, Estimating the number of unseen species: How many words did Shakespeare know?, addresses related issues.

In the following, you'll need do two things -
- replace all the low-frequency words (frequency = 1, 2, 5, or some other value) in the training data with the token `<UNK>`. You may try 1 to start. 
- use add-lambda smoothing (lambda = 1 in add-one smoothing) where you can let lambda be 1 or 0.001. You may try .001 to start. 


In [ ]:
#TODO: replace all the low-frequent words in the training data with the token <UNK>.
# the rest will be similar to the part right preceeding part 1. 
# also report the size of the vocabulary and set the value of lambda to prepare for add-lambda smoothing. 
lambda_value = 0.00001

def replace_low_freq(sentence):
    low_freq = 1

    words_UNK = [word if unigram_fdist[word] > low_freq else "<UNK>" for word in sentence.split()]
  
    return " ".join(words_UNK)

sentences_UNK = [replace_low_freq(sent) for sent in sentences]

for sent in sentences_UNK[0:10]:
    print(sent, end="\n")


def train_ngram_smoothed(ngrams, vocab):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    # vocab = len(set(ngrams)) # make vocab number of N-1 grams

    for ng in ngrams:
        model[ng[:-1]][ng[-1]] += 1  # [(w1, w2, ... , wn-1)][wn]
    
    total_count = 0      
    for hist in model: # hist = (w1, w2, ... , wn-1)
        total_count = float(sum(model[hist].values()))
        # if "<UNK>" in hist:
        #     total_count = 0
        for w in model[hist]: # w = wn
            count = model[hist][w]
            # if w == "<UNK>":
            #     count = 0
            model[hist][w] = (count + lambda_value)/(total_count + lambda_value * vocab)
    
    return model

<s> liberty all star usa sets initial payout </s>
<s> we are being accused of not implementing this agreement </s>
<s> entregrowth closed at 135 dlrs and options at 55 cents </s>
<s> usda forecast south african 1986 87 corn exports at 210 mln tonnes vs 300 mln tonnes last month and 1985 86 exports at 275 mln tonnes vs 275 mln tonnes last month </s>
<s> <UNK> issued capital will be <UNK> mln shares of which 63 pct will be held by nbh after 89 mln are issued to shareholders to raise 196 mln dlrs it said </s>
<s> the april 6 sale to be evenly divided between the three and six month issues will result in a paydown of 165 billion dlrs as maturing bills total 1485 billion dlrs </s>
<s> waste managements tender offer announced before the opening today expires march 25 </s>
<s> he earlier estimated the damage from the us raid at about 500 mln dlrs </s>
<s> brougher bigi to sell 40 pct of subsidiary </s>
<s> that was not the case two years ago </s>


**3.1 Extract smoothed unigrams from training data**

In [ ]:
#Todo: extract the smoothed unigram model. same as part 1.1 except for the smoothed probability
unigrams_UNK = [ug for sent in sentences_UNK for ug in sent.split()]
unigrams_count_UNK = len(unigrams_UNK)
unigrams_UNK_vocab = len(set(unigrams_UNK))

unigram_fdist_UNK = nltk.FreqDist(unigrams_UNK)

unigram_model_smoothed = {}
for ug, freq in unigram_fdist_UNK.most_common(200): 
    if ug == "<UNK>":
        freq = 0
    
    unigram_model_smoothed[ug] = (freq + lambda_value)/(unigrams_count_UNK + lambda_value * unigrams_UNK_vocab)

**3.2 Extract smoothed bigrams from training data**

In [ ]:
#Todo: extract the smoothed bigram model. same as part 1.2 except for the smoothed probability

bigrams_UNK = [bg for sent in sentences_UNK for bg in nltk.bigrams(sent.split())]
bigrams_UNK_vocab = len(set(bigrams_UNK))

bigram_model_smoothed = train_ngram_smoothed(bigrams_UNK,unigrams_UNK_vocab)

**3.3 Extract smoothed trigrams from training data**

In [ ]:
#Todo: extract the smoothed trigram model, same as part 1.3 except for the smoothed probability

#re-padding each sentence with <s><s> at the beginning 
#that is, append extra <s> to the beginning of each sentence

trigrams_UNK = [tg for sent in sentences_UNK for tg in nltk.trigrams(["<s>"] + sent.split())]
trigrams_UNK_vocab = len(set(bigrams_UNK))

trigram_model_smoothed = train_ngram_smoothed(trigrams_UNK,bigrams_UNK_vocab)


# **Task 4 - Random sentence generator using smoothed n-grams**

**4.1 Generate 10 sentences (with probabilities) using the smoothed unigram model**

In [ ]:
unigram_sentences = []

#generate the sentences
for index in range(0,num_sentences):
    test, probability = unigram_random_sentence(unigram_model_smoothed)

    unigram_sentences.append((test,probability))
    
#TODO: generate 10 sentences using the smoothed unigram model. Suppose maximum sentence length = 30
    
print_sentences(unigram_sentences)

<s> if profit loan exchange company export while we west through about tax this japan into months loan securities japanese stock nine yen share japan new 4 were up told by </s> 
probability of:  9.456286023975352e-93



<s> than after its shr to rate inc and banks government expected trade from foreign billion could had or common between 30 we because been but american earnings finance not made </s> 
probability of:  2.486760097080471e-87



<s> has had was rates time more between told at while vs dlr four tonnes been investment three because group world government foreign capital financial february spokesman after share dollar and </s> 
probability of:  3.1754295172463096e-90



<s> company at net world international months this rate years would price bank cash tax dollar for major at its three 30 year export profit for vs total some to or </s> 
probability of:  1.051098633164143e-84



<s> exchange this over it credit february from spokesman 30 shrs offer april not <UNK> exports doll

**4.2 Generate 10 sentences (with probabilities) using the smoothed bigram model**

In [ ]:
bigram_sentences = []

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(bigram_model_smoothed, ["<s>"])

    bigram_sentences.append((test,probability))

#ToDo: generate 10 sentences using the smoothed bigram model. Suppse maximum sentence length = 30

print_sentences(bigram_sentences)

<s> total us oil traders and the acquisition that he still being placed its economy continues to be for which fell 02 nigeria and industry output expected improvement in october 14 </s> 
probability of:  3.4446681060629033e-60



<s> an indicated as at five g under federal trade policy decisions bosworth said i did succeed we know who has also rose to go star gets 52 pct higher us </s> 
probability of:  7.576873342923105e-61



<s> i see these matters might come different origin due between seven pretax net assets 45 tonnes 1986 a group including those issued earlier new crop report it believes subsequent repayments </s> 
probability of:  8.363363096616448e-65



<s> japan were likely mean he cited tax income tax rate currently provides </s> 
probability of:  8.561472143295514e-29



<s> national wildlife in response she sees second qtr excludes 625000 dlr contract expiry of japan officials point purolator courier companies operating next autumn intervention bd says trade data voice hi

**4.3 Generate 10 sentences (with probabilities) using the smoothed trigram model**

In [ ]:
trigram_sentences = []

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(trigram_model_smoothed, ["<s>","<s>"])

    trigram_sentences.append((test,probability))

#ToDo: generate 10 sentences using the smoothed trigram model. Suppse maximum sentence length = 30
    
print_sentences(trigram_sentences)

<s> <s> us home resales up in court settlement </s> 
probability of:  1.49967398887116e-10



<s> <s> national business review newspaper published in june 10 </s> 
probability of:  3.417181547416186e-12



<s> <s> first connecticut has assets of this has increasingly failed to bridge the funding gap </s> 
probability of:  1.9402094455712195e-19



<s> <s> american powdered makes metal parts for various sept dec same section 125 pct while real imports had risen almost three dollars below a target companys stock that will have </s> 
probability of:  5.650583488141795e-31



<s> <s> new homes </s> 
probability of:  3.866469509864546e-06



<s> <s> for discontinued compressor operations vs loss 35 cts net 1523000 vs 2557000 dlrs in new manufacturing plant targeted for sale and or canadian transshipment points by september 20 the </s> 
probability of:  3.11766149234365e-33



<s> <s> we may not occur but it hasnt already agreed to stimulate growth and huge trade imbalances it welcomed the f

# **Task 5 - Evaluate your language models**

Computing the perplexity of the test data for the three smoothed language models. An introduction to perplexity can be found on page 8 of [Chapter 3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) of Jurasky and Martin's book. 

As an example, for bigram, the perplexity of a corpus W with N words is calculated as shown [here](https://drive.google.com/file/d/1r-WGzQxKiecJ_vlGqiYEryLkZajEMQTO/view?usp=sharing).

In the following, report the perplexities for the 3 smoothed language models on the test data set. 
     

**5.0 - Process the test data**

In [ ]:
# TODO: 
# Replace all the "unknown" words in the test set (that is, the words that are not in the vocabulary that is exacted from the training data) with the token "`<UNK>`". 
# And put the test data in a list to get ready for n-gram model extraction. 

def replace_unknown(sentence):
    words_UNK = [word if unigram_fdist[word] == 0 else "<UNK>" for word in sentence.split()]
  
    return " ".join(words_UNK)


test_sentences = load_file(test_file)

test_sentences_UNK = [replace_unknown(sent) for sent in test_sentences]

# i = 0
# for sent, test_sent in zip(test_sentences, test_sentences_UNK):
#     if "<UNK>" in test_sent:
#         print(sent)
#         print(test_sent,"\n")
#         i += 1

#     if i > 10:
#         break

**5.1 - Calculate and report the perplexity of the unigram model**

In [ ]:
#TODO: calculate the perplexity of the smoothed unigram model on the test data
# report the perplexity 

test_unigrams_UNK = [ug for sent in test_sentences_UNK for ug in sent.split()]

ln_prob_sum = 0
for ug in test_unigrams_UNK:
    freq = unigram_fdist_UNK[ug]
    # if ug == "<UNK>":
    #     freq = 0
    
    ln_prob_sum += math.log((freq + lambda_value)/(unigrams_count_UNK + lambda_value * unigrams_UNK_vocab))


test_unigrams_perplexity = math.exp((-1/len(test_unigrams_UNK)) * ln_prob_sum)

print(test_unigrams_perplexity)

94.35421100479897


**5.2 - Calculate and report the perplexity of the bigram model**

In [ ]:
#TODO: calculate the perplexity of the smoothed bigram model on the test data
# report the perplexity 

test_bigrams_UNK = [bg for sent in test_sentences_UNK for bg in nltk.bigrams(sent.split())]
test_bigrams_UNK_vocab = len(set(test_bigrams_UNK))

ln_prob_sum = 0
for bg in test_bigrams_UNK:
    prob = bigram_model_smoothed[bg[:-1]][bg[-1]]
    if bg[1] == "<s>":
        continue
    if prob == 0: 
        total_count = float(sum(bigram_model_smoothed[bg[:-1]].values()))
        prob = (0 + lambda_value)/(total_count + lambda_value * unigrams_UNK_vocab)
    ln_prob_sum += math.log(prob)

test_bigrams_perplexity = math.exp((-1/len(test_bigrams_UNK)) * ln_prob_sum)

print(test_bigrams_perplexity)

32.234066219728085


**5.3 - Calculate and report the perplexity of the trigram model**

In [ ]:
#TODO: calculate the perplexity of the smoothed bigram model on the test data
# report the perplexity 
#don't forget to append extra <s> to the beginning of each sentence


test_trigrams_UNK = [tg for sent in test_sentences_UNK for tg in nltk.trigrams(["<s>"] + sent.split())]
test_trigrams_UNK_vocab = len(set(test_trigrams_UNK))

ln_prob_sum = 0
for tg in test_trigrams_UNK:
    prob = trigram_model_smoothed[tg[:-1]][tg[-1]]
    if tg[2] == "<s>" or (tg[1] == "<s>" and tg[2] == "<s>"):
        continue
    if prob == 0: 
        total_count = float(sum(trigram_model_smoothed[(tg[0],)].values()))
        prob = (0 + lambda_value)/(total_count + lambda_value * bigrams_UNK_vocab)
    
    ln_prob_sum += math.log(prob)

test_trigrams_perplexity = math.exp((-1/len(test_trigrams_UNK)) * ln_prob_sum)

print(test_trigrams_perplexity)

12.426922831929732


# **Task 6 - Summary and Reflection** 

Summarize what you've learnt for this assignment and the challenges you've run into and tackled. Report any future work and possible improvement you would make. 

I learned a lot about python dictionaries as well as the technicalities of implementing ngram language models.  That sample code I found in the link provided in Task 1 was really invaluable for helping me think through the implementations; it help me structure and generalize my code a lot.  

One of the biggest difficulties I had with this assignment was figuring out the initial implementations of the different ngram models.  Once I figured out how that worked, a lot of the math and other implementations just fell into place.  I also had some trouble with the math of the smoothed models just becuase I misunderstood how we were supposed to count the \<UNK\> tokens.  I initially implemented the models to count tokens that contained \<UNK\> as having zero occurances in order for their probabilites to be really low after smoothing. This seemed to work fine until Task 5 where my smoothed models produced some really strange results for the perplexities, giving incredibly large perplexity values.  Counting \<UNK\> normally lead to much less strange results.  

# **Extra: Most Probable Sentences**

In [ ]:
def print_sentences_joined(sent_list):
    for sent, prob in sent_list:
        print(sent)
        print("probability of: ", prob)
        print("\n\n")

## **Unsmoothed Models**

**Unigram sentences**

In [ ]:
unigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = unigram_random_sentence(unigram_model)

    unigram_sentences.append((test,probability))

i = num_of_sents
most_probable_unigrams = []
base_prob = unigram_model["<s>"] * unigram_model["</s>"]
for ug, freq in heapq.nlargest(12, unigram_model.items(), key=operator.itemgetter(1)):
    if ug == "<s>" or ug == "</s>":
        continue
    
    most_probable_unigrams.append((["<s>",ug,"</s>"], freq * base_prob))


unigram_sentences = [(" ".join(words), prob) for words, prob in unigram_sentences + most_probable_unigrams]
unigram_sentences_most_probable = heapq.nlargest(num_sentences, set(unigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(unigram_sentences_most_probable)

<s> </s>
probability of:  0.0016710226810651028



<s> the </s>
probability of:  7.7529927625609e-05



<s> of </s>
probability of:  4.132424835468951e-05



<s> to </s>
probability of:  3.975543425383651e-05



<s> to </s>
probability of:  3.9755434253836505e-05



<s> in </s>
probability of:  3.092658566739602e-05



<s> and </s>
probability of:  2.9999869645556873e-05



<s> said </s>
probability of:  2.8122529694681258e-05



<s> a </s>
probability of:  2.809634484394133e-05



<s> mln </s>
probability of:  1.9494052139989764e-05





**Bigram sentences**

In [ ]:
bigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = ngram_random_sentence(bigram_model, ["<s>"],n=2)

    bigram_sentences.append((test,probability))


bigram_sentences = [(" ".join(words), prob) for words, prob in bigram_sentences]
bigram_sentences_most_probable = heapq.nlargest(num_sentences, set(bigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(bigram_sentences_most_probable)

<s> it said </s>
probability of:  0.0013780822775624668



<s> the company said </s>
probability of:  0.0006643501238030103



<s> the company </s>
probability of:  0.0005298434729746154



<s> it said it said </s>
probability of:  3.291486556938212e-05



<s> the company said it said </s>
probability of:  1.586769917298153e-05



<s> the us agriculture department said </s>
probability of:  8.143811253345884e-06



<s> the us agriculture committee </s>
probability of:  9.184014946733082e-07



<s> it said it said it said </s>
probability of:  7.86156525695098e-07



<s> it has a share </s>
probability of:  6.029778334990044e-07



<s> it has a year </s>
probability of:  4.0929087602791655e-07





**Trigram sentences**

In [ ]:
trigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = ngram_random_sentence(trigram_model, ["<s>","<s>"],n=2)

    trigram_sentences.append((test,probability))


trigram_sentences = [(" ".join(words), prob) for words, prob in trigram_sentences]
trigram_sentences_most_probable = heapq.nlargest(num_sentences, set(trigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(trigram_sentences_most_probable)

<s> <s> it said </s>
probability of:  0.004516589861751152



<s> <s> the company said </s>
probability of:  0.0020537495799109857



<s> <s> the company </s>
probability of:  0.0010262692255342315



<s> <s> the bank said </s>
probability of:  0.0003509761941700712



<s> <s> the bank of england said </s>
probability of:  1.7690021842244334e-05



<s> <s> it said the company said </s>
probability of:  1.2972386842340124e-05



<s> <s> it said the company </s>
probability of:  6.482368409587638e-06



<s> <s> the bank of japan governor haruo maekawa </s>
probability of:  1.6280381563479762e-06



<s> <s> it said the us agriculture department said </s>
probability of:  1.4561112739651348e-06



<s> <s> the company said it will be listed in luxembourg </s>
probability of:  6.164617200998113e-07





## **Smoothed Models**

**Unigram sentences**

In [ ]:
unigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = unigram_random_sentence(unigram_model_smoothed)

    unigram_sentences.append((test,probability))

i = num_of_sents
most_probable_unigrams = []
base_prob = unigram_model_smoothed["<s>"] * unigram_model_smoothed["</s>"]
for ug, freq in heapq.nlargest(12, unigram_model_smoothed.items(), key=operator.itemgetter(1)):
    if ug == "<s>" or ug == "</s>":
        continue
    
    most_probable_unigrams.append((["<s>",ug,"</s>"], freq * base_prob))


unigram_sentences = [(" ".join(words), prob) for words, prob in unigram_sentences + most_probable_unigrams]
unigram_sentences_most_probable = heapq.nlargest(num_sentences, set(unigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(unigram_sentences_most_probable)

<s> </s>
probability of:  0.0016710221464266586



<s> the </s>
probability of:  7.75298904158908e-05



<s> of </s>
probability of:  4.132422852684377e-05



<s> to </s>
probability of:  3.975541517915787e-05



<s> in </s>
probability of:  3.0926570831331966e-05



<s> in </s>
probability of:  3.092657083133196e-05



<s> and </s>
probability of:  2.9999855254397043e-05



<s> said </s>
probability of:  2.8122516204807753e-05



<s> a </s>
probability of:  2.8096331366638827e-05



<s> mln </s>
probability of:  1.9494042792535337e-05





**Bigram sentences**

In [ ]:
bigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = ngram_random_sentence(bigram_model_smoothed, ["<s>"],n=2)

    bigram_sentences.append((test,probability))


bigram_sentences = [(" ".join(words), prob) for words, prob in bigram_sentences]
bigram_sentences_most_probable = heapq.nlargest(num_sentences, set(bigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(bigram_sentences_most_probable)

<s> it said </s>
probability of:  0.0013780328124173638



<s> the company said </s>
probability of:  0.0006642968180612163



<s> the company </s>
probability of:  0.0005298060134788247



<s> it said it said </s>
probability of:  3.291263149833008e-05



<s> the company said it said </s>
probability of:  1.586591856256916e-05



<s> the us agriculture department said </s>
probability of:  8.137147368641228e-06



<s> the us agriculture committee </s>
probability of:  9.174829959369047e-07



<s> it said it said it said </s>
probability of:  7.860780254169949e-07



<s> it has a share </s>
probability of:  6.028753240826936e-07



<s> it has a year </s>
probability of:  4.092416948332726e-07





**Trigram sentences**

In [ ]:
trigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = ngram_random_sentence(trigram_model_smoothed, ["<s>","<s>"],n=2)

    trigram_sentences.append((test,probability))


trigram_sentences = [(" ".join(words), prob) for words, prob in trigram_sentences]
trigram_sentences_most_probable = heapq.nlargest(num_sentences, set(trigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(trigram_sentences_most_probable)

<s> <s> it said </s>
probability of:  0.004496529521320802



<s> <s> the company said </s>
probability of:  0.002043716639246769



<s> <s> the company </s>
probability of:  0.0010243482456935642



<s> <s> the bank said </s>
probability of:  0.00034549887440574493



<s> <s> the bank of england said </s>
probability of:  1.6589274228118742e-05



<s> <s> it said the company said </s>
probability of:  1.284371903484674e-05



<s> <s> it said the company </s>
probability of:  6.437507435656652e-06



<s> <s> it said the us agriculture department said </s>
probability of:  1.3739909232705634e-06



<s> <s> the company said it will be listed in luxembourg </s>
probability of:  5.631226728508644e-07



<s> <s> the bank of japan governor haruo maekawa </s>
probability of:  1.325993441202779e-07





# **Grading Rubric**
This assignment was worth 60 points total. The rubic used for grading this homework is below.

**Part 0 - Process the training data (5 points)**

**Part 1 - 3 language models (10 points)**
- -34 if no probabilities are reported

**Part 2 - random sentence generation (15 points)**
- -6 if no probabilities are reported

**Part 3 - handling unknown tokens and smoothing n-gram models(7 points)**
- -3 smoothing is not done
- -4 unknown tokens are not handled.

**Part 4 - sentence generation (5 points)**

**Part 5 - perplexity for 3 language models (15 points)**

**Part 6 - summary and reflection (3 points)** 


**Extra Credit (10 points max)**

- +10 generate the 10 most probable sentences (instead of random sentences) for each of the 3 language models



# Resources

I used the idea of this function to make my models.  It came from the link included at the beginning of Task 1.

In [ ]:
# MIT License

# Copyright (c) 2018 Gaurav Bhatt

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

def train_bigram(lst):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for sent in lst:
        sent = sent.split()
        for w1, w2 in bigrams(sent, pad_right=True, pad_left=True):
            model[w1][w2] += 1  
    total_count = 0      
    for w1 in model:
        total_count = float(sum(model[w1].values()))
        for w2 in model[w1]:
            model[w1][w2] /= total_count
    return model
